 # Algoritmo genético

Los algoritmos genéticos funcionan iterando sobre generaciones de poblaciones y evaluando qué tan bien estas poblaciones resuelven un problema. Al final de la generación, se seleccionan los mejores individuos para producir la próxima generación.

## Abejas vs Pesticidas
 
El uso excesivo de agroquímicos en los campos está comprometiendo el futuro de las abejas, desencadenando consecuencias que pueden ir más allá de los aspectos ambientales pues estos insectos polinizan los principales cultivos del mundo. Se estima que el 75 por ciento de la alimentación humana depende directa o indirectamente de la acción de los insectos polinizadores.

### Conceptualización

Un pesticida es un compuesto químico que tiene la importancia para el control de plagas en la agricultura. Debido a lo anterior los pesticidas no solamente matan plagas de insectos que afectan los cultivos, sino que también poblaciones de abejas melíferas, las cuales benefician el medio ambiente.

Las abejas son los mayores polinizadores de muchas plantas silvestres y monocultivo. Motivo por el cual, su población es fundamental para la productividad agrícola mundial y es evidente que las alteraciones en sus poblaciones podrían derivar en significativas pérdidas económicas.

La situación actual de las abejas es preocupante, debido a que el censo poblacional ha disminuido drásticamente por el uso extensivo de pesticidas en monocultivos tecnificados que buscan potenciar el rendimiento de la producción.

Los pesticidas son sustancias producidas de forma natural o industrial para destruir organismos potencialmente perjudiciales para las plantas cultivadas y/o animales de producción. Así mismo, se clasifican de acuerdo con el tipo de peste que controlan en: insecticidas (contra insectos), herbicidas (flora arvense), rodenticidas (roedores), bactericidas (bacterias), fungicidas (hongos) y larvicidas (larvas).

Estas sustancias pueden ser tóxicas a su vez para las especies que comparten el sitio de fumigación y, dependiendo de la dosificación, pueden tener una acción letal o subletal; a la par, permanecer residualmente en el medio ambiente.

### Planteamiento

Con base a lo anterior un pesticida contempla tres propiedades inherentes, las cuales son composición, forma de aplicación y residualidad:

**Composición:** Está compuesto por varios químicos incluyendo un ingrediente activo que es el químico que mata, controla o repele plagas.

**Exposición:** La exposición de las abejas se puede presentar por contacto, por alimentación o por las dos situaciones combinadas.

**Aplicación:** Las formas de aplicación de los pesticidas con sus respectivas características, son espolvoreo, pulverización, fumigación, aplicación de cebos, tratamientos vía riego y aplicación en el suelo.

**Residuality:** La residualidad del pesticida es un resultado de diversos factores, como la forma y sitio de aplicación, dosis, grado de infestación de la plaga y nivel de control o riesgo, exposición a microorganismos y materia orgánica.

Teniendo en cuenta las características mencionadas de los pesticidas, se desea saber cómo deben combinarse estas cuatro características para saber el grado de toxicidad potencial del pesticida y así minimizar la tasa de mortalidad de las poblaciones de abejas melíferas. Para llegar a este resultado hay que tener presente que:

* El porcentaje de ingrediente activo en el compuesto puede ser variable.
* Algunos pesticidas son: Corrosivos, irritantes, inflamables o explosivos.
* La forma de aplicación puede ser cualquiera de las mencionadas anteriormente.
* La residualidad  depende de las condiciones de la aplicación.
* La exposición de las abejas a los pesticidas puede ser por contacto, alimentación o ambas.
* El grado de toxicidad potencial del ingrediente activo puede ser: De baja peligrosidad, nocivo, toxico, muy tóxico.

La idea es que el modelo se pueda aplicar para cualquier combinación de cultivo-plaga


# Librerías

In [199]:
import numpy as np
import random
import operator
import pandas as pd
import matplotlib.pyplot as plt

# Representación de los datos

Los **cromosomas** del individuo para encontrar una posible solución al problema de las Abejas vs Pesticidas, estan representados pos las siguientes clases:

* Cromosoma **Ingrediente** define el ingrediente activo utilizado para un determinado cultivo (Lista de posibles ingredientes activos utilizados para un determinado cultivo) presente en el compuesto y respectiva clasificación (Corrosivo, irritante, inflamable o explosivo).
* Cromosoma **Exposicion** define si la exposición que presentan las abejas mielíferas al ingreduente activo es por contacto, alimentación o ambas. 
* Cromosoma **Aplicacion** define la forma en la que es aplicado el pesticida en el cultivo (espolvoreo, pulverización, fumigación, aplicación de cebos, tratamientos vía riego y aplicación en el suelo)
* Cromosoma **Residualidad** define el tiempo en días que el ingrediente activo esta presente en el ecosistema

Cada uno de los **individuos** del algoritmo genético esta representado por la clase **Combinacion**, la cual es la unión de las clases anteriormente mencionadas más el porcentaje de concentración del ingrediente activo:

* Clase **Compuesto** define el nombre del ingrediente activo presente en el compuesto

In [200]:
class Ingrediente:
    def __init__(self, ingrediente = None, clasificacion = []):
        self.ingrediente = ingrediente
        self.clasificacion = clasificacion

class Exposicion:
    def __init__(self, forma = None):
        self.forma = forma

class Aplicacion:
    def __init__(self, tipo = None):
        self.tipo = tipo

class Residualidad:
    def __init__(self, tiempo = None):
        self.tiempo = tiempo

class Combinacion:
    def __init__(self, concentracion, ingrediente = Ingrediente(), exposicion = Exposicion(), aplicacion = Aplicacion(), residualidad = Residualidad()):
        self.concentracion = concentracion
        self.exposicion = exposicion
        self.ingrediente = ingrediente
        self.aplicacion = aplicacion
        self.residualidad = residualidad  

# ¿Cual es el problema a resolver?

El objetivo es calcular la toxicidad potencial de cada individuo por la combinación de la concentración de un ingrediente activo específico, la forma de exposición, el tipo de aplicación y la residualidad  para cualquier cultivo en el cual se aplique algún tipo de pesticida, vamos a simular un caso particular para un cultivo dado, aplicando varios posibles pesticidas especializados en el cultivo seleccionado.

## Problema

La simulación 

In [201]:
clasificacion = {
  1: 1, # Corrosivo
  2: 2, # Irritante
  3: 3, # Inflamable
  4: 4 # Explosivo
}

exposicion = {
  1: 1, # Contacto
  2: 2, # Alimentación
  3: 3 # Ambas
}

aplicacion = {
  1: 1, # Espolvoreo
  2: 2, # Contactopulverización
  3: 3, # Fumigación
  4: 4, # Aplicación de cebos
  5: 5, # Tratamientos vía riego
  6: 6 # Aplicación en el suelo
}

residualidad = {
  1: 1,
  5: 2,
  10: 3,
  15: 4,
  20: 5,
  25: 6,
  30: 7,
  35: 8,
  40: 9,
  45: 10,
  50: 11,
  55: 12,
  60: 13,
  65: 14,
  70: 15,
  75: 16,
  80: 17,
  85: 18,
  90: 19
}

## Aptitud
Para este caso, la aptitud de un individuo es el calculo de la toxicidad del mismo que esta dado por la combinación de la concentración de un ingrediente activo específico, la forma de exposición, el tipo de aplicación y la residualidad como se menciono anteriormente y dicha aptitud esta dada por la formula:

>$Aptitud=RC+[\sum_{i=1}^{N} RCI(Valor_{i})]+[\sum_{i=1}^{N} RFE(Valor_{i})]+[\sum_{i=1}^{N} RFA(Valor_{i})]+[\sum_{i=1}^{N} RTR(Valor_{i})]$

>$RC = 1/(1,1 - concentración)$

### Descripción

>**RC** = Riesgo por concentración 

>**RCI** = Riesgo por clasificación del ingrediente activo

>**RFE** = Riesgo por forma de exposición

>**RFA** = Riesgo por forma de aplicación

>**RTR** = Riesgo por tiempo de residualidad

Ya que  es importante definir una función de aptitud común para todos los individuos, está formula fue construida con base a la **tabla 1**. La función hace el sumatorio de puntos de riesgo acumulados por cada uno de los individuos y calcula la inversa de para definir la toxicidad potencial de cada individuo.

In [202]:
class Aptitud:
    def __init__(self, combinacion):
        self.combinacion = combinacion
        self.riesgo = 0
        self.aptitud= 0.0
    
    def sumaAptitud(self):
        if self.riesgo ==0:
            suma = 1/ (1.1 - self.combinacion.concentracion)
            if self.combinacion.ingrediente.clasificacion in clasificacion:
                suma += clasificacion[self.combinacion.ingrediente.clasificacion]
            if self.combinacion.exposicion.forma in exposicion:
                suma += exposicion[self.combinacion.exposicion.forma]
            if self.combinacion.aplicacion.tipo in aplicacion:
                suma += aplicacion[self.combinacion.aplicacion.tipo]
            if self.combinacion.residualidad.tiempo in residualidad:
                suma += residualidad[self.combinacion.residualidad.tiempo]
            self.riesgo = suma
        return self.riesgo
    
    def calculoAptitud(self):
        if self.aptitud == 0:
            self.aptitud = 1 / float(self.sumaAptitud())
        return self.aptitud

## Mutación y reproducción
El algoritmo crea las próximas generaciones utilizando dos métodos, ya sea mutando individuos individuales según una probabilidad, o "apareando" dos individuos para crear uno nuevo. Aquí definimos todas las funciones necesarias para realizar el algoritmo.

In [203]:
#Creamos la población inicial de forma aleatoría.
def poblacionInicial(cantidad):
    poblacion = []
    for i in range(0,cantidad):
        poblacion.append(Combinacion(
            concentracion=random.random(),
            ingrediente=Ingrediente("Ingrediente activo " + str(i + 1), random.randint(1, 4)),
            exposicion=Exposicion(random.randint(1, 3)),
            aplicacion=Aplicacion(random.randint(1, 6)),
            residualidad=Residualidad(random.randint(1, 90))
            )
        )
    return poblacion

#Ordena la población deacuerdo a su aptitud
def organizarAptitudes(poblacion):
    aptitudes = {}
    for i in range(0,len(poblacion)):
        aptitudes[i] = Aptitud(poblacion[i]).calculoAptitud()
    aptitudes_ordenadas=sorted(aptitudes.items(), key = operator.itemgetter(1), reverse = True)
    return aptitudes_ordenadas

#Creamos la función de selección
def seleccion(aptitudesOrdenadas, seleccionElitista):
    resultados = []
    df = pd.DataFrame(np.array(aptitudesOrdenadas), columns=["Index","Aptitud"])
    df['cum_sum'] = df.Aptitud.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Aptitud.sum()
    
    for i in range(0, seleccionElitista):
        resultados.append(aptitudesOrdenadas[i][0])
    for i in range(0, len(aptitudesOrdenadas) - seleccionElitista):
        pick = 100*random.random()
        for i in range(0, len(aptitudesOrdenadas)):
            if pick <= df.iat[i,3]:
                resultados.append(aptitudesOrdenadas[i][0])
                break
    return resultados

#Crear vector de apareamiento
def apareamiento(poblacion, seleccionados):
    aparear = []
    for i in range(0, len(seleccionados)):
        index = seleccionados[i]
        aparear.append(poblacion[index])
    return aparear

#Crea dos hijos a partir de dos padres
def cruce(padre1, padre2, tasaCruce):
    hijos = []
    hijo1 = Combinacion(
            concentracion=padre1.concentracion,
            ingrediente=padre1.ingrediente,
            exposicion=padre1.exposicion,
            aplicacion=padre1.aplicacion,
            residualidad=padre1.residualidad
            )
    hijo2 = Combinacion(
            concentracion=padre2.concentracion,
            ingrediente=padre2.ingrediente,
            exposicion=padre2.exposicion,
            aplicacion=padre2.aplicacion,
            residualidad=padre2.residualidad
            )
    if random.random() < tasaCruce:
        hijo1.concentracion = padre2.concentracion
        hijo2.concentracion = padre1.concentracion
    if random.random() < tasaCruce:
        hijo1.ingrediente.clasificacion = padre2.ingrediente.clasificacion
        hijo2.ingrediente.clasificacion = padre1.ingrediente.clasificacion
    if random.random() < tasaCruce:
        hijo1.exposicion.forma = padre2.exposicion.forma
        hijo2.exposicion.forma = padre1.exposicion.forma
    if random.random() < tasaCruce:
        hijo1.aplicacion.tipo = padre2.aplicacion.tipo
        hijo2.aplicacion.tipo = padre1.aplicacion.tipo
    if random.random() < tasaCruce:
        hijo1.residualidad.tiempo = padre2.residualidad.tiempo
        hijo2.residualidad.tiempo = padre1.residualidad.tiempo
    hijos.append(hijo1)
    hijos.append(hijo2)
    return hijos

#Crear función para cruzar toda la población
def cruzarPoblacion(matingpool, seleccionElitista, tasaCruce):
    hijos = []
    length = len(matingpool) - seleccionElitista
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,seleccionElitista):
        hijos.append(matingpool[i])

    for i in range(0, length):
        child = cruce(pool[i], pool[len(matingpool)-i-1], tasaCruce)
        hijos.append(child[0])
        hijos.append(child[1])
    return hijos

#Crea función de mutación
def mutacion(individuo, tasaMutacion):
    if random.random() < tasaMutacion:
        individuo.concentracion = random.random()
    if random.random() < tasaMutacion:
        individuo.ingrediente.clasificacion = random.randint(1, 4)
    if random.random() < tasaMutacion:
        individuo.exposicion.forma = random.randint(1, 3)
    if random.random() < tasaMutacion:
        individuo.aplicacion.tipo = random.randint(1, 6)
    if random.random() < tasaMutacion:
        individuo.residualidad.tiempo = random.randint(1, 90)
    return individuo

#Mutamos toda la población
def mutarPoblacion(poblacion, tasaMutacion):
    mutados = []
    for i in range(0, len(poblacion)):
        modificado = mutacion(poblacion[i], tasaMutacion)
        mutados.append(modificado)
    return mutados

#Crea una nueva generación
def nuevaGeneracion(generacionActual, seleccionElitista, tasaMutacion, tasaCruce):
    aptitudesOrdenadas = organizarAptitudes(generacionActual)
    seleccionados = seleccion(aptitudesOrdenadas, seleccionElitista)
    aparear = apareamiento(generacionActual, seleccionados)
    children = cruzarPoblacion(aparear, seleccionElitista, tasaCruce)
    cruce = mutarPoblacion(children, tasaMutacion)
    return cruce

## El algoritmo genético
Programamos el algoritmo genético con las funciones previas.

In [204]:
def algoritmo(poblacion, seleccionElitista, tasaMutacion, tasaCruce, generaciones):
    progress = [1 / organizarAptitudes(poblacion)[0][1]]

    pop = poblacion
    for i in range(1, generaciones+1):
        print("Generación: ")
        pop = nuevaGeneracion(pop, seleccionElitista, tasaMutacion, tasaCruce)
        progress.append(1 / organizarAptitudes(pop)[0][1])
        if i%50==0:
          print('Generation '+str(i),"Distance: ",progress[i])
        
    mejorIndividuoIndex = organizarAptitudes(pop)[0][0]
    mejorIndividuo = pop[mejorIndividuoIndex]
    
    plt.plot(progress)
    plt.ylabel('Riesgo')
    plt.xlabel('Generation')
    plt.title('Mejor aptitud vs Generación')
    plt.tight_layout()
    plt.show()

    
    
    return mejorIndividuo

## Pruebas
Realizamos las pruebas del algoritmo genetico previamente programado.

In [ ]:
inicial=poblacionInicial(40)
solucion=algoritmo(poblacion=inicial, seleccionElitista=20, tasaMutacion=0.01, tasaCruce=0.05, generaciones=10)
print("****************** Solución *************************")
print(solucion.concentracion)
print(solucion.ingrediente.clasificacion)
print(solucion.exposicion.forma)
print(solucion.aplicacion.tipo)
print(solucion.residualidad.tiempo)

****************** 0 *************************
0.6797938496210767
3
2
5
76
****************** 1 *************************
0.2258958780028575
2
2
4
27
****************** 2 *************************
0.21524398068334671
2
3
1
79
****************** 3 *************************
0.25893828653683315
2
2
3
15
****************** 4 *************************
0.10473696831135049
1
1
6
5
****************** 5 *************************
0.506980115614934
3
1
1
15
****************** 6 *************************
0.44080735967893303
3
2
5
17
****************** 7 *************************
0.800755295487153
2
3
2
9
****************** 8 *************************
0.165915158855362
4
2
3
77
****************** 9 *************************
0.6395426435336855
3
3
5
61
****************** 10 *************************
0.38180081852709113
2
3
1
62
****************** 11 *************************
0.8516300614404708
3
1
1
49
****************** 12 *************************
0.5328038421796246
3
2
4
69
****************** 13 

## Investigaciones y Artículos

[1] [Guía de Evaluación de Riesgos de Plaguicidas para las Abejas](https://espanol.epa.gov/sites/production-es/files/2017-12/documents/120517_guia_de_evaluacion_de_riesgos_de_plaguicidas_para_las_abejas_update.pdf)

[2] [Abejas y Agrotóxicos](https://www.apiservices.biz/documents/articulos-es/abejas_y_agrotoxicos.pdf)

[3] [Los plaguicidas, adsorción y evolución en el suelo](https://digital.csic.es/bitstream/10261/12919/1/plaguicidas.pdf%3B)

[4] [Calculo de la toxicidad potencial de los plaguicidas](https://www.tdx.cat/bitstream/handle/10803/6827/06CAPITOL5.pdf?sequence=6&isAllowed=y)

[5] [Desarrollo de métodos para la reducción de la contaminación por plaguicidas en aguas subterráneas mediante la adición de residuos orgánicos a los suelos](https://idus.us.es/bitstream/handle/11441/41630/Tesis%20completa%20(enviar).pdf)

[6] [Daño colateral en abejas por la exposición a pesticidas de uso agrícola](http://www.scielo.org.co/pdf/entra/v14n1/1900-3803-entra-14-01-232.pdf)

[7] [Insecticidas más tóxico para las abejas ¿Por qué?](https://agriculturers.com/insecticidas-mas-toxico-para-las-abejas-por-que/)

[8] [Residualidad de un herbicida](https://www.rainbowconosur.com/uy/detalle-de-residualidad-de-un-herbicida-194)

